# Attempting XLNet with side information
Testing to see if it can work with continous features as well. 
Starting with only segment repetition

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
base_path = Path("~/fall_project/MOOCCubeX/")
results_path = Path("./results")
relations_path = base_path / "relations"
session2video_id_path = relations_path / "sessions_w_concepts"

In [3]:
%%time
sessions = pd.read_parquet(session2video_id_path)

CPU times: user 30 s, sys: 3.36 s, total: 33.4 s
Wall time: 33.6 s


In [4]:
sessions

,user_id,video_consecutive_id,session_id,seg_rep_count,video_id,local_start_time,ccid,concept_ids,concepts,fields,interaction_session
0,U_1002476,1,0,0,V_1353266,1598971366,D8EC8658CD3FC45D9C33DC5901307461,"[K_边界的受力_物理学, K_变量_物理学, K_单形_物理学, K_动量矩方程_物理学,...","[边界的受力, 变量, 单形, 动量矩方程, 观察, 轨道, 阶, 理论力学, 力的平衡，,...","[物理学, 物理学, 物理学, 物理学, 物理学, 物理学, 物理学, 物理学, 物理学, ...",U_1002476-1-0
1,U_1002476,2,0,0,V_6258948,1601736654,1D4D20B05FD0D8B99C33DC5901307461,"[K_包含已知直线_机械工程, K_辅助平面法_机械工程, K_辅助平面法求直线和平面交点_...","[包含已知直线, 辅助平面法, 辅助平面法求直线和平面交点, 辅助平面, 积聚性, 几何元素...","[机械工程, 机械工程, 机械工程, 机械工程, 机械工程, 机械工程, 机械工程, 机械工...",U_1002476-2-0
2,U_1002476,3,0,0,V_6258951,1601736952,292B73EF985AE70A9C33DC5901307461,"[K_z坐标轴_机械工程, K_包含已知直线_机械工程, K_侧面投影_机械工程, K_辅助...","[z坐标轴, 包含已知直线, 侧面投影, 辅助平面法求直线和平面交点, 辅助平面, 几何学原...","[机械工程, 机械工程, 机械工程, 机械工程, 机械工程, 机械工程, 机械工程, 机械工...",U_1002476-3-0
3,U_10027765,1,0,1,V_7704153,1601717800,2C72189286FF3EF99C33DC5901307461,[],[],[],U_10027765-1-0
4,U_10027765,2,0,1,V_8186213,1604113273,DC81374DA06F06D89C33DC5901307461,[],[],[],U_10027765-2-0
...,...,...,...,...,...,...,...,...,...,...,...
2009151,U_9999820,1,0,0,V_7386531,1601434095,1BAE71E966D980C29C33DC5901307461,[],[],[],U_9999820-1-0
2009152,U_9999820,2,0,0,V_7386532,1601451341,AD99106BD3AD4A789C33DC5901307461,[],[],[],U_9999820-2-0
2009153,U_9999820,3,0,0,V_7386534,1601468512,8D21D8DA741CBD639C33DC5901307461,[],[],[],U_9999820-3-0
2009154,U_9999820,4,0,0,V_7386535,1601474220,81FDDCC818CF26339C33DC5901307461,[],[],[],U_9999820-4-0


In [5]:
ITEM_COL = "video_id"
USER_COL = "user_id"
CONTINUOUS_COL = "seg_rep_count"
TIMESTAMP_COL = "local_start_time"

In [6]:
video_id2int = {val: i for i, val in enumerate(sessions[ITEM_COL].unique(), start=1)}
int2video_id = {i: video_id for video_id, i in video_id2int.items()}

In [7]:
sessions[ITEM_COL].nunique()

185637

In [8]:
MAX_VIDEO_ID = max(video_id2int.values())

In [9]:
# Basic video_id, encoding, saving 0 for padding
sessions_encoded = sessions.copy()
sessions_encoded.loc[:,ITEM_COL] = sessions_encoded[ITEM_COL].map(video_id2int)

In [10]:
# Functions for pre processing
def minmax_norm(series):
    if not series.any():
        return series
    return ((series - series.min())/(series.max() - series.min())).fillna(0)
def adaptive_z_score(series):
    """Adaptively standardizes the segment repetition as one learns more about the user's repetition behaviour.
    First repetition is deemed as 0. Alternative, lookinto it as 1"""
    if not series.any():
        return series
    expanding_window = series.expanding(1)
    return ((series - expanding_window.mean())/expanding_window.std()).fillna(0)

In [11]:
%%time
# Pre-process and group to lists
sessions_list = (sessions_encoded.sort_values(TIMESTAMP_COL)
                     .groupby(USER_COL)
                     .agg({ITEM_COL: list, 
                           CONTINUOUS_COL: lambda series: minmax_norm(series).tolist()}))
sessions_list_hot_5 = sessions_list[sessions_list[ITEM_COL].str.len() >= 5]
#.apply(pre_process, seg_rep_fn=adaptive_z_score).groupby("user_id").agg(list)

CPU times: user 27 s, sys: 315 ms, total: 27.3 s
Wall time: 27.5 s


In [12]:
sessions_list_hot_5

,video_id,seg_rep_count
user_id,,
U_10001587,"[11923, 11993, 11924, 11924, 11924, 12024]","[0.0, 1.0, 0.0, 0.0, 0.25, 0.0]"
U_10008027,"[12082, 34059, 12145, 12031, 12036, 11924]","[0, 0, 0, 0, 0, 0]"
U_10012383,"[12147, 12143, 12082, 12080, 34059, 12145, 121...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
U_10018910,"[2766, 2762, 2763, 2764, 2765, 2768]","[0, 0, 0, 0, 0, 0]"
U_100203,"[142499, 142500, 87277, 125176, 125177, 142500...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
U_9979639,"[2769, 2766, 2762, 2763, 2764, 2768]","[0, 0, 0, 0, 0, 0]"
U_998508,"[25718, 2221, 2222, 2223, 2953, 2953, 28783, 7...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
U_9988528,"[160890, 160891, 160891, 160892, 160892, 16089...","[0.3333333333333333, 0.0, 0.3333333333333333, ..."


In [13]:
sessions_list_hot_5[ITEM_COL].str.len().describe(percentiles=np.arange(.5, 1, 0.05))

count    114150.000000
mean         14.279877
std          15.113597
min           5.000000
50%           9.000000
55%          10.000000
60%          11.000000
65%          12.000000
70%          14.000000
75%          16.000000
80%          18.000000
85%          23.000000
90%          30.000000
95%          44.000000
max         570.000000
Name: video_id, dtype: float64

In [14]:
MAX_SEQUENCE_LENGTH = 50
pad_token = 0

In [15]:
def slice_and_pad(series):
    return series.transform(lambda lst: (lst + [pad_token] * MAX_SEQUENCE_LENGTH)[:MAX_SEQUENCE_LENGTH])

In [16]:
# Slice and pad with 0
sessions_list_h5_sliced = sessions_list_hot_5.transform(slice_and_pad, axis=0)

In [17]:
raw_lists = np.array(sessions_list_h5_sliced[ITEM_COL].tolist())
raw_lists

array([[ 11923,  11993,  11924, ...,      0,      0,      0],
       [ 12082,  34059,  12145, ...,      0,      0,      0],
       [ 12147,  12143,  12082, ...,      0,      0,      0],
       ...,
       [160890, 160891, 160891, ...,      0,      0,      0],
       [  2769,   2766,   2762, ...,      0,      0,      0],
       [  2769,   2766,   2763, ...,      0,      0,      0]])

In [18]:
# First encountered zero
target_idx = (raw_lists==0).argmax(1, keepdims=True) - 1 
test_values = raw_lists

In [19]:
# Fill in the value which is hidden
# ->Split into train, validation and test with last and second-to-last items

val_values = test_values.copy()
np.put_along_axis(val_values, target_idx, pad_token, axis=1)
train_values = val_values.copy()
train_mask_idx = target_idx - 1 
np.put_along_axis(train_values, train_mask_idx, pad_token, axis=1)

In [24]:
test = sessions_list_h5_sliced#.to_frame()
val = pd.Series(list(val_values), index=sessions_list_h5_sliced.index, name=ITEM_COL)#.to_frame()
train = pd.Series(list(train_values), index=sessions_list_h5_sliced.index, name=ITEM_COL)#.to_frame()

In [25]:
base_df = sessions_list_h5_sliced[[CONTINUOUS_COL]]
test_df = base_df.merge(test, on=USER_COL)
val_df = base_df.merge(val, on=USER_COL)
train_df = base_df.merge(train, on=USER_COL)

In [26]:
val_df

,seg_rep_count,video_id
user_id,,
U_10001587,"[0.0, 1.0, 0.0, 0.0, 0.25, 0.0, 0, 0, 0, 0, 0,...","[11923, 11993, 11924, 11924, 11924, 0, 0, 0, 0..."
U_10008027,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[12082, 34059, 12145, 12031, 12036, 0, 0, 0, 0..."
U_10012383,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[12147, 12143, 12082, 12080, 34059, 12145, 121..."
U_10018910,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2766, 2762, 2763, 2764, 2765, 0, 0, 0, 0, 0, ..."
U_100203,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[142499, 142500, 87277, 125176, 125177, 142500..."
...,...,...
U_9979639,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2769, 2766, 2762, 2763, 2764, 0, 0, 0, 0, 0, ..."
U_998508,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[25718, 2221, 2222, 2223, 2953, 2953, 28783, 7..."
U_9988528,"[0.3333333333333333, 0.0, 0.3333333333333333, ...","[160890, 160891, 160891, 160892, 160892, 16089..."


In [27]:
train_df

,seg_rep_count,video_id
user_id,,
U_10001587,"[0.0, 1.0, 0.0, 0.0, 0.25, 0.0, 0, 0, 0, 0, 0,...","[11923, 11993, 11924, 11924, 0, 0, 0, 0, 0, 0,..."
U_10008027,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[12082, 34059, 12145, 12031, 0, 0, 0, 0, 0, 0,..."
U_10012383,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[12147, 12143, 12082, 12080, 34059, 12145, 121..."
U_10018910,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2766, 2762, 2763, 2764, 0, 0, 0, 0, 0, 0, 0, ..."
U_100203,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[142499, 142500, 87277, 125176, 125177, 142500..."
...,...,...
U_9979639,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2769, 2766, 2762, 2763, 0, 0, 0, 0, 0, 0, 0, ..."
U_998508,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[25718, 2221, 2222, 2223, 2953, 2953, 28783, 7..."
U_9988528,"[0.3333333333333333, 0.0, 0.3333333333333333, ...","[160890, 160891, 160891, 160892, 160892, 16089..."


In [28]:
out_path = Path("data-seg-rep")

In [29]:
test_df.to_parquet(out_path / "test.parquet")
val_df.to_parquet(out_path / "val.parquet")
train_df.to_parquet(out_path / "train.parquet")

### Create manual schema

In [30]:
from merlin_standard_lib import Schema, Tag, ColumnSchema
from merlin.schema.tags import Tags
from merlin_standard_lib.proto.schema_bp import ValueCount, IntDomain

In [31]:
schema = Schema([
        ColumnSchema.create_categorical(ITEM_COL, MAX_VIDEO_ID, 
                                        value_count= ValueCount(0, MAX_SEQUENCE_LENGTH),
                                        min_index=0,
                                        tags=[Tag.CATEGORICAL, Tag.LIST, Tag.ITEM, Tag.ITEM_ID]
                                       ),
        ColumnSchema.create_continuous(CONTINUOUS_COL, 
                                       min_value=0, max_value=1, 
                                       value_count=ValueCount(0, MAX_SEQUENCE_LENGTH), 
                                       tags=[Tag.CONTINUOUS, Tag.LIST])
    ])

In [173]:
with open(out_path/"schema.pb", "w") as file:
    file.write(schema.to_proto_text())

# XLNet Config and such

In [32]:
import os

#os.environ["CUDA_VISIBLE_DEVICES"]="0"
import glob
import torch 

from transformers4rec import torch as tr
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt, RecallAt
from transformers4rec.torch.utils.examples_utils import wipe_memory
from transformers4rec.config.transformer import BertConfig

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
schema

[{'name': 'video_id', 'value_count': {'max': '50'}, 'type': 'INT', 'int_domain': {'name': 'video_id', 'max': '185637', 'is_categorical': True}, 'annotation': {'tag': ['categorical', 'item', 'list', 'item_id']}}, {'name': 'seg_rep_count', 'value_count': {'max': '50'}, 'type': 'FLOAT', 'float_domain': {'name': 'seg_rep_count', 'max': 1.0}, 'annotation': {'tag': ['continuous', 'list']}}]

In [34]:
INTERACTION_EMB=64

In [35]:
# Define input module - No continous features -> No need for projection 
input_module = tr.TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length=MAX_SEQUENCE_LENGTH,
        #continuous_projection=16, #Project continous values before merging
        masking="mlm",
        d_output=INTERACTION_EMB, # Make output embedding match the embedding used in the transformer arch
)
input_module

TabularSequenceFeatures(
  (to_merge): ModuleDict(
    (continuous_module): ContinuousFeatures(
      (filter_features): FilterFeatures()
    )
    (categorical_module): SequenceEmbeddingFeatures(
      (filter_features): FilterFeatures()
      (embedding_tables): ModuleDict(
        (video_id): Embedding(185638, 64, padding_idx=0)
      )
    )
  )
  (_aggregation): ConcatFeatures()
  (projection_module): SequentialBlock(
    (0): DenseBlock(
      (0): Linear(in_features=65, out_features=64, bias=True)
      (1): ReLU(inplace=True)
    )
  )
  (_masking): MaskedLanguageModeling()
)

In [46]:
# Define XLNetConfig class and set default parameters for HF XLNet config  
transformer_config = tr.XLNetConfig.build(
    d_model=INTERACTION_EMB, n_head=4, n_layer=2, total_seq_length=MAX_SEQUENCE_LENGTH
)

metrics = [NDCGAt(top_ks=[5, 10], labels_onehot=True),  
           RecallAt(top_ks=[5, 10], labels_onehot=True),
           AvgPrecisionAt(top_ks=[5, 10], labels_onehot=True)
          ]

prediction_task = tr.NextItemPredictionTask(
    weight_tying=True, metrics=metrics, loss=torch.nn.NLLLoss(ignore_index=0)
)
model = transformer_config.to_torch_model(input_module, prediction_task)
model

Model(
  (heads): ModuleList(
    (0): Head(
      (body): SequentialBlock(
        (0): TabularSequenceFeatures(
          (to_merge): ModuleDict(
            (continuous_module): ContinuousFeatures(
              (filter_features): FilterFeatures()
            )
            (categorical_module): SequenceEmbeddingFeatures(
              (filter_features): FilterFeatures()
              (embedding_tables): ModuleDict(
                (video_id): Embedding(185638, 64, padding_idx=0)
              )
            )
          )
          (_aggregation): ConcatFeatures()
          (projection_module): SequentialBlock(
            (0): DenseBlock(
              (0): Linear(in_features=65, out_features=64, bias=True)
              (1): ReLU(inplace=True)
            )
          )
          (_masking): MaskedLanguageModeling()
        )
        (1): TansformerBlock(
          (transformer): XLNetModel(
            (word_embedding): Embedding(1, 64)
            (layer): ModuleList(
             

In [37]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
import random as pyrandom

In [49]:
#Setting all seeds, maybe redundant
seed = 53
torch.manual_seed(seed)
np.random.default_rng(seed)
pyrandom.seed(seed)

# Set hyperparameters for training 
TRAIN_BATCH_SIZE = 512
EVAL_BATCH_SIZE = 512
train_args = T4RecTrainingArguments(data_loader_engine='merlin', 
                                    per_device_train_batch_size = TRAIN_BATCH_SIZE,
                                    per_device_eval_batch_size = EVAL_BATCH_SIZE,
                                    output_dir = "./xlnet-seg-rep", 
                                    save_total_limit=5,
                                    logging_steps=100,
                                    report_to = [],
                                    num_train_epochs=20,
                                    max_sequence_length=MAX_SEQUENCE_LENGTH, 
                                    no_cuda=False,
                                    seed=seed,
                                    #save_strategy="steps",
                                    evaluation_strategy="steps",
                                    eval_steps=250,
                                    #save_steps=100,
                                    #load_best_model_at_end=True,
                                    # Optimizer - Adapted from Keras Adam default params
                                    learning_rate=0.001,
                                    lr_scheduler_type='cosine', 
                                    learning_rate_num_cosine_cycles_by_epoch=1.5,
                                   )
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
    train_dataset_or_path = str(out_path / "train.parquet"),
    eval_dataset_or_path = str(out_path / "val.parquet")
)

PyTorch: setting up devices


In [50]:
# INITIAL EVAL
trainer.evaluate()

{'eval_/next-item/ndcg_at_5': 0.014775990508496761,
 'eval_/next-item/ndcg_at_10': 0.02561997063457966,
 'eval_/next-item/recall_at_5': 0.028935611248016357,
 'eval_/next-item/recall_at_10': 0.06202365458011627,
 'eval_/next-item/avg_precision_at_5': 0.01020762138068676,
 'eval_/next-item/avg_precision_at_10': 0.014760906808078289,
 'eval_/loss': 9.849635124206543,
 'eval_runtime': 8.8516,
 'eval_samples_per_second': 12898.931,
 'eval_steps_per_second': 25.193}

In [51]:
trainer.train()

***** Running training *****
  Num examples = 114176
  Num Epochs = 20
  Instantaneous batch size per device = 512
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 1
  Total optimization steps = 4460


Step,Training Loss,Validation Loss,/next-item/ndcg At 5,/next-item/ndcg At 10,/next-item/recall At 5,/next-item/recall At 10,/next-item/avg Precision At 5,/next-item/avg Precision At 10,/loss
100,8.904100,No log,0.013464,0.030733,0.027832,0.080657,0.008906,0.016152,9.594467
200,8.549400,No log,0.014008,0.037139,0.028647,0.100526,0.009371,0.018857,9.176056
300,8.263900,No log,0.018359,0.047053,0.038143,0.126824,0.012084,0.023932,9.014377
400,7.939300,No log,0.027361,0.057227,0.056969,0.148296,0.018005,0.030526,8.870076
500,7.764700,No log,0.080917,0.096265,0.131336,0.177941,0.064299,0.070795,8.710693
600,7.640100,No log,0.084185,0.099360,0.142681,0.188682,0.064858,0.071296,8.656776
700,7.494600,No log,0.097790,0.112851,0.162111,0.208445,0.076562,0.082823,8.582703
800,7.411800,No log,0.122427,0.135236,0.188629,0.228191,0.100373,0.105668,8.482122
900,7.308600,No log,0.125704,0.139125,0.195664,0.237092,0.102370,0.107924,8.458697
1000,7.214700,No log,0.138693,0.151223,0.200193,0.238528,0.118201,0.123450,8.417460


Saving model checkpoint to ./xlnet-seg-rep/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./xlnet-seg-rep/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./xlnet-seg-rep/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./xlnet-seg-rep/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./xlnet-seg-rep/checkpoint-2500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [xlnet-seg-rep/checkpoint-100] due to args.save_total_limit
Saving model checkpoint to ./xlnet-seg-rep/checkpoint-3000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [xlnet-seg-rep/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to ./xlnet-seg-rep/checkpoint-3

TrainOutput(global_step=4460, training_loss=6.508838054845151, metrics={'train_runtime': 654.9653, 'train_samples_per_second': 0.031, 'train_steps_per_second': 6.81, 'total_flos': 0.0, 'train_loss': 6.508838054845151})